We are going to explore if it is reasonable to use a MLR model to predict the number of people graduating from three different levels of education. 
We will also explore if the model performs better for the entire population or for men and women separately.

In [ ]:
# d <- pxweb_interactive("https://api.scb.se/OV0104/v1/doris/sv/ssd/UF/UF0550/UF0550C/Historisk11bN")
API_wait <- 1e-3
pxq_edu <-
  list(
    "Examen" = c("*"), # Use "*" to select all
    "Kon" = c("*"),
    "ContentsCode" = c("000004NE"),
    "Tid" = c("*")
  )
pxg_edu <- pxweb_get("https://api.scb.se/OV0104/v1/doris/sv/ssd/START/UF/UF0550/UF0550C/Historisk11bN", pxq_edu)
Sys.sleep(API_wait)
pxq_pop <-
  list(
    "Kon" = c("*"),
    "ContentsCode" = c("000000LV"),
    "Tid" = c("*")
  )

pxg_pop <- pxweb_get("https://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101G/BefUtvKon1749", pxq_pop)
Sys.sleep(API_wait)
# https://api.scb.se/OV0104/v1/doris/sv/ssd/START/HE/HE0103/HE0103A/ArbInk28
pxq_income <- list(
  "Kon" = c("*"),
  "Tid" = c("*"),
  "ContentsCode" = c("HE0103CL")
  # "HE0103CM")
)
pxg_income <- pxweb_get("https://api.scb.se/OV0104/v1/doris/sv/ssd/START/HE/HE0103/HE0103A/ArbInk28", pxq_income)

Sys.sleep(API_wait)
pxq_kpi <- list(
  "Tid" = c("*"),
  "ContentsCode" = c("000004VU")
)
pxg_kpi <- pxweb_get("https://api.scb.se/OV0104/v1/doris/sv/ssd/START/PR/PR0101/PR0101A/KPItotM", pxq_kpi)


We start by gathering data from SCB about education,income,KPI and population.
Then we clean and modify the data to fit our needs.

In [ ]:
edu_df <- as.data.frame(pxg_edu)
edu_df <- as_tibble(edu_df)

doktor_df <- edu_df %>%
    filter(examen %in% c("Doktorsgrad", "Doktorsexamen")) %>%
    group_by(läsår, kön) %>%
    mutate(`Utfärdade examina vid universitet och högskolor` = sum(`Utfärdade examina vid universitet och högskolor`)) %>%
    filter(examen == "Doktorsexamen") %>%
    ungroup()


licenciat_df <- edu_df %>%
    filter(examen %in% c("Licentiatexamen", "Licentiatexamen (Äldre)")) %>%
    group_by(läsår, kön) %>%
    mutate(`Utfärdade examina vid universitet och högskolor` = sum(`Utfärdade examina vid universitet och högskolor`)) %>%
    filter(examen == "Licentiatexamen") %>%
    ungroup()


edu_df <- edu_df %>%
    filter(examen == "Examen från grundutbildning") %>%
    full_join(licenciat_df) %>%
    full_join(doktor_df) %>%
    pivot_wider(names_from = kön, values_from = `Utfärdade examina vid universitet och högskolor`) %>%
    mutate(läsår = as.integer(str_extract(läsår, "^\\w*"))) %>%
    rename(år = läsår) %>%
    pivot_longer(!c(år, examen), names_to = "kön", values_to = "antal") %>%
    mutate(
        kön = str_replace(kön, "båda könen", "totalt"),
        examen = factor(examen,
            ordered = FALSE,
            levels = c(
                "Examen från grundutbildning", "Licentiatexamen", "Licentiatexamen (Äldre)", "Doktorsgrad", "Doktorsexamen"
            ),
            labels = c(
                "Grundutbildning", "Licentiatexamen", "Licentiatexamen", "Doktorsexamen", "Doktorsexamen"
            )
        )
    )

pop_df <- as.data.frame(pxg_pop)
pop_df <- as_tibble(pop_df) %>% mutate(år = as.integer(år)) # %>%



income_df <- as.data.frame(pxg_income)
income_df <- as_tibble(income_df) %>%
    filter(år >= 1936) %>%
    mutate(år = as.integer(år))


kpi_df <- as.data.frame(pxg_kpi)
kpi_df <- as_tibble(kpi_df) %>%
    mutate(månad = str_extract(månad, "(20|19)[\\d]{2}")) %>%
    rename("år" = "månad") %>%
    group_by(år) %>%
    mutate(år = as.integer(år)) %>%
    summarize(`AVG_KPI_Fastställd` = mean(`KPI, fastställda tal`)) %>%
    ungroup()


final_df <- edu_df %>%
    left_join(income_df, by = join_by("år", kön == kön)) %>%
    left_join(kpi_df, by = "år") %>%
    left_join(pop_df, by = join_by("år", kön == kön)) %>%
    rename(medelinkomst = `Medelvärde, tkr`) %>%
    mutate(kön = factor(kön, levels = c("totalt", "kvinnor", "män"), ordered = FALSE)) %>%
    drop_na()

final_df %>% filter(år == 1980)


Joining with `by = join_by(examen, kön, läsår, `Utfärdade examina vid
universitet och högskolor`)`
Joining with `by = join_by(examen, kön, läsår, `Utfärdade examina vid
universitet och högskolor`)`


examen,år,kön,antal,medelinkomst,AVG_KPI_Fastställd,Folkmängd
<fct>,<int>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
Grundutbildning,1980,totalt,34536,172.3,100,8317937
Grundutbildning,1980,män,12110,220.4,100,4119822
Grundutbildning,1980,kvinnor,22426,122.9,100,4198115
Licentiatexamen,1980,totalt,1,172.3,100,8317937
Licentiatexamen,1980,män,1,220.4,100,4119822
Licentiatexamen,1980,kvinnor,0,122.9,100,4198115
Doktorsexamen,1980,totalt,812,172.3,100,8317937
Doktorsexamen,1980,män,666,220.4,100,4119822
Doktorsexamen,1980,kvinnor,146,122.9,100,4198115


We then do some eploratory data analysis on the collected data

We can see that there is some collinearity between some of the columns and that it differs between different types of degrees.

As a naive approach the first model will contain all available data in the multiple linear regression for the amount of graduates.

In [ ]:
naive_model2 <- lm(antal ~ år + kön + examen + medelinkomst + AVG_KPI_Fastställd, data = final_df %>% filter(kön != "totalt"))
summary(naive_model2)


This naive model results in a relatively high $R^2$ and $R_{adj}^{2}$ values of $R^2=0.836, R_{adj}^{2}=0.830$. The most significant regressors were the average KPI that year and if the degree was a "Licenciat" or PH. That the type of degree was significant is not at all unexpected as a Bachelor's level degree is a prerequesite for further studies. That KPI was significant could hint at the state of the economy being important in people studying or not. But the model is quite hard to interpret as a person, because of the large amount of explanatory variables. Because of this the regressor with the highest $p$-value was removed from the model, this being the population variable.

The new model had $R^2=0.836, R_{adj}^{2}=0.831$ meaning it performed slightly better than the naive model. Altough it should be noted that all of the $p$-values barely changed.

In order to increase the interpretability of the model and to more easily choose our variables, we will instead create two new models one for the amount of people graduating with a Bachelor's level degree and another for the amount of people graduating with either a "Licenciat" or PHD.

We will also aproach the problem of choosing variables in the opposite way from what was done to the naive model, that is we will add one variable at a time instead of removing them.


In [ ]:
bachelor_model <- lm(antal ~ ., data = final_df %>% filter(examen == "Grundutbildning", kön != "totalt") %>% select(!examen))
ols_step_forward_p(bachelor_model)

higher_ed_model <- lm(antal ~ ., data = final_df %>% filter(examen != "Grundutbildning", kön != "totalt") %>% mutate(examen = factor(examen)))
ols_step_forward_p(higher_ed_model)


The variable selection algorithm for the Bachelor's level model resulted in the following choices:
1. Population
2. Gender
3. Average income
4. Average KPI
5. Year

On first glance this result was surprising to me as the first variable it chose was population in contrast to the naive model were it had the largest $p$-values, and that the variable with the lowest $p$-value except type of degree was added second to last.

Altough from the pairs plot we knew that there was collinearity between some of the variables so it is to be excpected that they would interact in nonintuitive ways. 

For the model of the other two degrees, forward selection resulted in the choices: